In [ ]:
print 'ok. automatic keyphrase extraction'

In [28]:
fileFPath='/home/mprakash/Projects/enron-experiments/significant-terms/output/firstpass/maildir/delainey-d/maildir_delainey-d_all_documents_72.'


In [29]:
f = open(fileFPath, 'r')
content = f.read()
f.close()
print 'File Content'
print content

File Content
Dorie, I have to question the effectiveness of this program.  Over the years, 
it appears that more spots are being filled by Enron people then customers. 
The effectiveness of this forum from a customer point of view is 
questionable.  I vote to kill it.

Please forgive the approach - but I would like to provoke a discussion with 
my partners.

Regards
Delainey
	
	
	From:  Dorie Hitchcock                           11/30/2000 06:40 AM
	

To: Sylvia S Pollan/HOU/ECT@ECT, Robyn Zivic/NA/Enron@Enron, Andrew H 
Lewis/HOU/ECT@ECT, Geoff Storey/HOU/ECT@ECT, Martin Cuilla/HOU/ECT@ECT, 
Patrice L Mims/HOU/ECT@ECT, Kevin Ruscitti/HOU/ECT@ECT, Kelli 
Stevens/HOU/ECT@ECT, Tom Donohoe/HOU/ECT@ECT, Joe Parks/Corp/Enron@ENRON, 
Janet H Wallis/HOU/ECT@ECT, Gary Bryan/HOU/ECT@ECT, Jill T 
Zivley/HOU/ECT@ECT, Brian Bierbach/NA/Enron@Enron, John Craig 
Taylor/HOU/ECT@ECT, Gary Hickerson/HOU/ECT@ECT, Daniel Reck/HOU/ECT@ECT, 
George McClellan/HOU/ECT@ECT, Mark Tawney/HOU/ECT@ECT, Jere C 
Ove

In [4]:
import spacy
nlp = spacy.load('en')

In [43]:
def np_drop_singletermcommonnoun(noun_chunk):
    tokens = [(token.orth_, token.pos_) for token in noun_chunk]
    if(len(tokens) == 1 and tokens[0][1] == 'NOUN'):
        return []
    else:
        return tokens
    
def np_drop_unwanted(noun_chunk):
    unwanted = ['SPACE', 'DET', 'NUM']
    stop_words = set('for a of the and to in what my you this is your where'.split())
    tokens = [(token.orth_.lower(), token.pos_) for token in noun_chunk 
                  if token.pos_ not in unwanted and token.orth_ not in stop_words]
    return tokens

def np_drop_typicalcommonwords(noun_chunk):
    common_words = set('information meeting review'.split())
    tokens = [(token.orth_.lower(), token.pos_) for token in noun_chunk 
                  if token.orth_ not in common_words]
    return tokens

    
    
doc = nlp(unicode(content, "utf-8"))
nchunks = doc.noun_chunks
print 'Candidate Keyphrases'
for nc in nchunks:
    print nc.lemma_
    print [(token.orth_, token.pos_) for token in nc]
    print 'preprocess_np_drop_unwanted', np_drop_unwanted(nc)
    print 'preprocess_np_drop_singletermcommonnoun', np_drop_singletermcommonnoun(nc)
    print 'preprocess_np_drop_typicalcommonwords', np_drop_typicalcommonwords(nc)
    print '------------------------------------------'
#     for token in nc:
#         print token.orth_, token.pos_
#         print 'xxxxxxxxxxxx'
    
print '-----------------------------'
# nounphrases = [[np.orth_, np.root.head.orth_] for np in doc.noun_chunks]
# print 'nounphrases with head', nounphrases
# print '-----------------------------'
# print 'Named Entities'
entities3 = doc.ents
for e in entities3:
    print e.text, e.label_ , [w.tag_ for w in e]

Candidate Keyphrases
the effectiveness
[(u'the', u'DET'), (u'effectiveness', u'NOUN')]
preprocess_np_drop_unwanted [(u'effectiveness', u'NOUN')]
preprocess_np_drop_singletermcommonnoun [(u'the', u'DET'), (u'effectiveness', u'NOUN')]
preprocess_np_drop_typicalcommonwords [(u'the', u'DET'), (u'effectiveness', u'NOUN')]
------------------------------------------
this program
[(u'this', u'DET'), (u'program', u'NOUN')]
preprocess_np_drop_unwanted [(u'program', u'NOUN')]
preprocess_np_drop_singletermcommonnoun [(u'this', u'DET'), (u'program', u'NOUN')]
preprocess_np_drop_typicalcommonwords [(u'this', u'DET'), (u'program', u'NOUN')]
------------------------------------------
the year
[(u'the', u'DET'), (u'years', u'NOUN')]
preprocess_np_drop_unwanted [(u'years', u'NOUN')]
preprocess_np_drop_singletermcommonnoun [(u'the', u'DET'), (u'years', u'NOUN')]
preprocess_np_drop_typicalcommonwords [(u'the', u'DET'), (u'years', u'NOUN')]
------------------------------------------
more spot
[(u'more', u'

In [16]:
import gensim
from collections import defaultdict, OrderedDict
# from pprints import pprint # pretty-printer
doc = nlp(unicode(content, "utf-8"))
nchunks = doc.noun_chunks
def generate_candidate_phrases():
    print 'generate_candidate_phrases'
    for nc in nchunks:
#         print nc.text.lower()
        yield nc.text.lower()
def score_keyphrases_by_tfidf(candidate_phrases):
    print 'score_keyphrases_by_tfidf'
    stoplist = set('for a of the and to in'.split())
    candidate_phrases = list(generate_candidate_phrases())
    texts = [[word.lower() for word in candidate_phrase.split() if word.lower() not in stoplist]
             for candidate_phrase in candidate_phrases]
#     frequency = defaultdict(int)
#     for text in texts:
#         for token in text:
#             frequency[token] += 1
#     texts2 = [[token for token in text ]
#                  for text in texts]
    print 'texts input to dictionary', texts
    
    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(aCandidatePhrase.split()) for aCandidatePhrase in candidate_phrases]
    tfidf = gensim.models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf, dictionary

doctfidf, docdictionary = score_keyphrases_by_tfidf(nchunks)
print 'tfidf of the corpus'

d = {docdictionary.get(id): value for doc in doctfidf for id, value in doc}
d_in_order = sorted(d.items(), key=lambda t: t[1], reverse=True)
# print d_in_order
for (name, value) in d_in_order:
    print name, value

    


score_keyphrases_by_tfidf
generate_candidate_phrases
texts input to dictionary [[u'guys'], [u'california', u'issues'], [u'everything'], [u'position'], [u'very', u'effective', u'manner'], [u'our', u'view'], [u'growth'], [u'iso', u'structures'], [u'necessary', u'evil'], [u'socio-political', u'leaning'], [u'these', u'entities'], [u'directors'], [u'these', u'boards'], [u'full', u'court', u'press'], [u'plan'], [u'place']]
tfidf of the corpus
growth 1.0
everything 1.0
place 1.0
plan 1.0
guys 1.0
directors 1.0
position 1.0
entities 0.8
boards 0.8
socio-political 0.707106781187
our 0.707106781187
issues 0.707106781187
structures 0.707106781187
evil 0.707106781187
california 0.707106781187
necessary 0.707106781187
iso 0.707106781187
leaning 0.707106781187
view 0.707106781187
these 0.6
manner 0.57735026919
court 0.57735026919
full 0.57735026919
very 0.57735026919
press 0.57735026919
effective 0.57735026919


In [17]:
from itertools import takewhile, tee, izip
import networkx

doc = nlp(unicode(content, "utf-8"))
nchunks = doc.noun_chunks
def generate_candidate_phrases():
    print 'generate_candidate_phrases'
    for nc in nchunks:
#         print nc.text.lower()
        yield nc.text.lower()
    
def score_keyphrases_by_textrank(text, n_keywords=0.1):
    
    
    # tokenize for all words, and extract *candidate* words
#     words = [word.lower()
#              for sent in nltk.sent_tokenize(text)
#              for word in nltk.word_tokenize(sent)]
    stoplist = 'for a of the and to in'.split()
    pronoun_list = 'what where this that which what whose his her our their'.split()
    candidate_phrases = generate_candidate_phrases()
    
    texts = [[word.lower() for word in candidate_phrase.split() if word.lower() not in set(stoplist+pronoun_list)]
             for candidate_phrase in candidate_phrases]
    print 'texts', texts
    words = [word.lower() for phrase in texts
                  for word in phrase]
    print 'words', words
    
    
    # build graph, each node is a unique candidate
    graph = networkx.Graph()
    graph.add_nodes_from(set(words))
    # iterate over word-pairs, add unweighted edges into graph
    def pairwise(iterable):
        """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
        a, b = tee(iterable)
        next(b, None)
        return izip(a, b)
    for w1, w2 in pairwise(words):
        if w2:
            print 'add edge for', sorted([w1, w2])
            graph.add_edge(*sorted([w1, w2]))
    # score nodes using default pagerank algorithm, sort by score, keep top n_keywords
    ranks = networkx.pagerank(graph)
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(words) * n_keywords))
        print 'calculated n_keywords', n_keywords
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.iteritems(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())
    print 'keywords', keywords
    # merge keywords into keyphrases
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            # counter as hackish way to ensure merged keyphrases are non-overlapping
            j = i + len(kp_words)
    
    return sorted(keyphrases.iteritems(), key=lambda x: x[1], reverse=True)

scored_keyphrases = score_keyphrases_by_textrank('junk')
print 'scored keyphrases are....'
for kw,score in scored_keyphrases:
    print kw, ':', score

generate_candidate_phrases
texts [[u'guys'], [u'california', u'issues'], [u'everything'], [u'position'], [u'very', u'effective', u'manner'], [u'view'], [u'growth'], [u'iso', u'structures'], [u'necessary', u'evil'], [u'socio-political', u'leaning'], [u'these', u'entities'], [u'directors'], [u'these', u'boards'], [u'full', u'court', u'press'], [u'plan'], [u'place']]
words [u'guys', u'california', u'issues', u'everything', u'position', u'very', u'effective', u'manner', u'view', u'growth', u'iso', u'structures', u'necessary', u'evil', u'socio-political', u'leaning', u'these', u'entities', u'directors', u'these', u'boards', u'full', u'court', u'press', u'plan', u'place']
add edge for [u'california', u'guys']
add edge for [u'california', u'issues']
add edge for [u'everything', u'issues']
add edge for [u'everything', u'position']
add edge for [u'position', u'very']
add edge for [u'effective', u'very']
add edge for [u'effective', u'manner']
add edge for [u'manner', u'view']
add edge for [u'gro